In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import sys
sys.path.append('D:\\Compute Science\\Machine Learning\\论文\\项目\\FairSPL\\venv_torch')
sys.path.append('D:\\Compute Science\\Machine Learning\\论文\\项目\\FairSPL\\venv_torch\\lib\\site-packages')

In [2]:
import json
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from dataset import fetch_data
from models.AdversarialDebiasing import AdversarialDebiasing
from eval import Evaluator
from utils import get_curriculum_stages
# from tmp.AdversarialDebiasing_copied2_before_lr import AdversarialDebiasing

In [3]:
data = fetch_data("adult")
print("data.x_train.shape: ", data.x_train.shape)
print("data.x_test.shape: ", data.x_test.shape)
origin_evaluator, train_evaluator, test_evaluator = Evaluator(data.s_train, "origin"), Evaluator(data.s_train,
                                                                                                 "train"), Evaluator(
    data.s_test, "test")
if data.s_val is not None:
    val_evaluator = Evaluator(data.s_val, "val")

n_features, n_classes, n_groups = data.x_train.shape[1], len(np.unique(data.y_train)), len(np.unique(data.s_train))
if n_classes == 2:
    n_classes = 1
if n_groups == 2:
    n_groups = 1

print("========== before train ==========")
origin_res = origin_evaluator(data.y_train, no_train=True)

Loading Adult dataset..
train_val_df.shape:  (32561, 14)
test_df.shape:  (12661, 14)


E:\Environment\python\python3.7.4\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Dataset statistic - #total: 45222; #train: 26049; #val.: 6512; #test: 12661; #dim.: 95

data.x_train.shape:  (26049, 95)
data.x_test.shape:  (12661, 95)
========== before train ==========
========== Results on origin ==========
Grp. 0 - #instance: 8460; #pos : 954
Grp. 1 - #instance: 17589; #pos : 5496
Demographic parity: 0.199702; Equal opportunity: 0.000000


In [4]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier(n_estimators=100, random_state=0)
dp = DemographicParity(difference_bound=0.001)
reduction = ExponentiatedGradient(classifier, dp)

reduction.fit(data.x_train, data.y_train, sensitive_features=data.s_train)

total step:  5


ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x00000239440156C8>,
                      estimator=GradientBoostingClassifier(random_state=0),
                      nu=0.0010570582632976704)

In [6]:
print("========== after train(without debiasing) ==========")
pred_label_train = reduction.predict(data.x_train)
train_res = train_evaluator(data.y_train, pred_label_train, no_train=False)

pred_label_test = reduction.predict(data.x_test)
test_res = test_evaluator(data.y_test, pred_label_test, no_train=False)

========== after train(without debiasing) ==========
========== Results on train ==========
Grp. 0 - #instance: 8460; #pos. pred: 1355; Acc.: 0.904137
Grp. 1 - #instance: 17589; #pos. pred: 2853; Acc.: 0.815510
Overall acc.: 0.844293; Demographic parity: 0.002038; Equal opportunity: 0.320778
========== Results on test ==========
Grp. 0 - #instance: 4147; #pos. pred: 652; Acc.: 0.895105
Grp. 1 - #instance: 8514; #pos. pred: 1323; Acc.: 0.809725
Overall acc.: 0.837691; Demographic parity: 0.001831; Equal opportunity: 0.285915


In [7]:
N = 3
stages = get_curriculum_stages(data.y_train, data.s_train, 'data/adult/sorted_idx_%d.json' % data.num_val, N=N)
# 从难到易
# stages.reverse()
# 并集
# new_stages = []
# for i in range(len(stages) - 1, 0, -1):
#     new_stages.append(np.concatenate([stages[i], stages[i - 1]]))

# new_stages.append(stages[0])
# new_stages.reverse()
# base_epoch = 500
# epoch_list = [int(base_epoch / i) for i in range(1,N+1)]
# epoch_list.reverse()

new_stages = [stages[0]]
for i in range(1, len(stages)):
    new_stages.append(np.concatenate([stages[i], new_stages[-1]]))

g:0, y:0 ==> size:2502
g:0, y:1 ==> size:318
g:1, y:0 ==> size:4031
g:1, y:1 ==> size:1832
g:0, y:0 ==> size:2502
g:0, y:1 ==> size:318
g:1, y:0 ==> size:4031
g:1, y:1 ==> size:1832
g:0, y:0 ==> size:2502
g:0, y:1 ==> size:318
g:1, y:0 ==> size:4031
g:1, y:1 ==> size:1832


In [9]:
# CL test
reduction_cl = ExponentiatedGradient(classifier, dp)
start = datetime.datetime.now()
for stage  in new_stages:
    # TODO: try set1 U set2
    print(len(stage))
    x_train = data.x_train[stage, :]
    y_train = data.y_train[stage]
    s_train = data.s_train[stage]
    reduction_cl.fit(x_train, y_train, sensitive_features=s_train)

end = datetime.datetime.now()
print((end - start).seconds)

8683


AssertionError: data can be loaded only once